In [4]:
#click on the right arrow

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import os

# Initialize WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

# Function to extract links from the current page
def extract_links_from_page():
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

# Open the initial URL
driver.get("https://www.myscheme.gov.in/search")

all_links = []

# Loop through each page and extract links
while True:
    all_links.extend(extract_links_from_page())
    try:
        next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(1)  # Wait for scrolling to complete
        next_button.click()
        time.sleep(2)  # Adjust the sleep time if needed to allow the next page to load properly
    except Exception as e:
        print("No more pages or error navigating to the next page. Stopping.")
        break
# Close the WebDriver
driver.quit()

print(f"Scraped {len(all_links)} links.")

No more pages or error navigating to the next page. Stopping.
Scraped 2439 links.


In [11]:
import nest_asyncio
nest_asyncio.apply()


In [72]:
#find the duplicate links in the list

def find_duplicates_with_indexes(lst):
    from collections import defaultdict

    counts = defaultdict(list)
    duplicates = []

    for index, item in enumerate(lst):
        counts[item].append(index)

    for item, indexes in counts.items():
        if len(indexes) > 1:
            duplicates.append((item, indexes))

    return duplicates


print(find_duplicates_with_indexes(all_links))

NameError: name 'all_links' is not defined

In [53]:
#Remove duplicates

def remove_duplicates(lst):
    seen = set()
    unique_links = []
    for item in lst:
        if item not in seen:
            unique_links.append(item)
            seen.add(item)
    return unique_links


unique_links = remove_duplicates(all_links_headless)
print(len(unique_links))


170


In [11]:
# Function to scrape data from each scheme page
def scrape_scheme_page(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
        detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
        benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
        eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
        exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
        application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
        documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
        faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
        keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
        
        return {
            'SchemeName': scheme_name,
            'Details': detail,
            'Benefits': benefit,
            'Eligibility': eligibility,
            'Exclusion': exclusion,
            'ApplicationProc': application_process,
            'DocumentsReq': documents_required,
            'FAQ': faq,
            'Link': url,
            'Keywords': keyword
        }
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# Load existing data if available
if os.path.exists('scheme_sample.csv'):
    existing_data = pd.read_csv('scheme_sample.csv')
    existing_links = set(existing_data['Link'])
    print('A')
else:
    existing_data = pd.DataFrame()
    existing_links = set()
    print('B')

# Initialize lists to store new data
new_data = []

# Scrape each URL and append data to lists
for url in unique_links:
    if url not in existing_links:
        sample = scrape_scheme_page(url)
        if sample:
            new_data.append(sample)

# Create a pandas DataFrame for new data
new_data_df = pd.DataFrame(new_data)

# Combine new data with existing data
if not existing_data.empty:
    combined_data = pd.concat([existing_data, new_data_df]).drop_duplicates(subset='Link', keep='last')
    print('a')
else:
    combined_data = new_data_df
    print('b')

# Save the combined data to CSV file
combined_data.to_csv('scheme_sample.csv', index=False)

print("Scraping and updating complete.")

B


KeyboardInterrupt: 

In [25]:
LINKS = unique_links


In [54]:
print(len(unique_links))

170


In [55]:
unique_links.sort()

In [56]:
#84 seconds but gives error
#157 seconds but no error
#143 seconds and no error

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Function to scrape data from each scheme page
def scrape_scheme_page(url):
    retries = 3
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code != 200:
                print(f"Failed to fetch {url} with status code {response.status_code}")
                return None
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
            detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
            benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
            eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
            exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
            application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
            documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
            faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
            keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
            
            return {
                'SchemeName': scheme_name,
                'Details': detail,
                'Benefits': benefit,
                'Eligibility': eligibility,
                'Exclusion': exclusion,
                'ApplicationProc': application_process,
                'DocumentsReq': documents_required,
                'FAQ': faq,
                'Link': url,
                'Keywords': keyword
            }
        except requests.RequestException as e:
            print(f"Request error scraping {url}: {e}")
        except Exception as e:
            print(f"Error scraping {url}: {e}")
        time.sleep(1)  # Sleep before retrying
    return None

def main():
    # Load existing data if available
    if os.path.exists('scheme_sample.csv'):
        existing_data = pd.read_csv('scheme_sample.csv')
        existing_links = set(existing_data['Link'])
    else:
        existing_data = pd.DataFrame()
        existing_links = set()

    # Initialize lists to store new data
    new_data = []

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scrape_scheme_page, url) for url in unique_links if url not in existing_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df]).drop_duplicates(subset='Link', keep='last')
    else:
        combined_data = new_data_df

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample.csv', index=False)

    print("Scraping and updating complete.")

if __name__ == "__main__":
    main()


Scraping and updating complete.


In [39]:
#64 seconds and no error
#50 seconds and no error
#53 seconds and no error
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Function to create a requests session with retries
def requests_session_with_retries(retries=3, backoff_factor=1):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(500, 502, 503, 504),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

# Function to scrape data from each scheme page
def scrape_scheme_page(session, url):
    try:
        response = session.get(url, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch {url} with status code {response.status_code}")
            return None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
        detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
        benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
        eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
        exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
        application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
        documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
        faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
        keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
        
        return {
            'SchemeName': scheme_name,
            'Details': detail,
            'Benefits': benefit,
            'Eligibility': eligibility,
            'Exclusion': exclusion,
            'ApplicationProc': application_process,
            'DocumentsReq': documents_required,
            'FAQ': faq,
            'Link': url,
            'Keywords': keyword
        }
    except requests.RequestException as e:
        print(f"Request error scraping {url}: {e}")
    except Exception as e:
        print(f"Error scraping {url}: {e}")
    return None

def main():
    # Load existing data if available
    if os.path.exists('scheme_sample2.csv'):
        existing_data = pd.read_csv('scheme_sample2.csv')
        existing_links = set(existing_data['Link'])
    else:
        existing_data = pd.DataFrame()
        existing_links = set()

    # Initialize lists to store new data
    new_data = []

    # Create a requests session with retries
    session = requests_session_with_retries()

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scrape_scheme_page, session, url) for url in unique_links if url not in existing_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df]).drop_duplicates(subset='Link', keep='last')
    else:
        combined_data = new_data_df

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample2.csv', index=False)

    print("Scraping and updating complete.")

if __name__ == "__main__":
    main()


Scraping and updating complete.


In [58]:
#trial with less data
#64 seconds and no error
#50 seconds and no error
#53 seconds and no error
#for 50 extra data, 3 seconds
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Function to create a requests session with retries
def requests_session_with_retries(retries=3, backoff_factor=1):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(500, 502, 503, 504),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

# Function to scrape data from each scheme page
def scrape_scheme_page(session, url):
    try:
        response = session.get(url, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch {url} with status code {response.status_code}")
            return None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
        detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
        benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
        eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
        exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
        application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
        documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
        faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
        keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
        
        return {
            'SchemeName': scheme_name,
            'Details': detail,
            'Benefits': benefit,
            'Eligibility': eligibility,
            'Exclusion': exclusion,
            'ApplicationProc': application_process,
            'DocumentsReq': documents_required,
            'FAQ': faq,
            'Link': url,
            'Keywords': keyword
        }
    except requests.RequestException as e:
        print(f"Request error scraping {url}: {e}")
    except Exception as e:
        print(f"Error scraping {url}: {e}")
    return None

def main():
    # Load existing data if available
    if os.path.exists('scheme_sample3.csv'):
        existing_data = pd.read_csv('scheme_sample3.csv')
        existing_links = set(existing_data['Link'])
    else:
        existing_data = pd.DataFrame()
        existing_links = set()

    # Initialize lists to store new data
    new_data = []

    # Create a requests session with retries
    session = requests_session_with_retries()

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scrape_scheme_page, session, url) for url in unique_links if url not in existing_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df]).drop_duplicates(subset='Link', keep='last')
    else:
        combined_data = new_data_df

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample3.csv', index=False)

    print("Scraping and updating complete.")

if __name__ == "__main__":
    main()


Scraping and updating complete.


In [45]:
import pandas as pd

def compare_csv_files(file1, file2):
    # Read the CSV files
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Check if the shapes of the dataframes are the same
    if df1.shape != df2.shape:
        return False

    # Check if the dataframes are equal
    return df1.equals(df2)

# Example usage
file1 = 'sorted_scheme_sample.csv'
file2 = 'sorted_scheme_sample2.csv'

if compare_csv_files(file1, file2):
    print("The contents of the two CSV files are the same.")
else:
    print("The contents of the two CSV files are different.")


FileNotFoundError: [Errno 2] No such file or directory: 'sorted_scheme_sample2.csv'

In [29]:

def compare_csv_files(file1, file2, max_differences=10):
    # Read the CSV files
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Check if the shapes of the dataframes are the same
    if df1.shape != df2.shape:
        print("The files have different shapes.")
        return

    # Check for differences
    differences = (df1 != df2)
    
    # Find differing cells
    differing_cells = differences.stack()
    differing_cells = differing_cells[differing_cells].index.tolist()
    
    total_differences = len(differing_cells)
    if total_differences == 0:
        print("The contents of the two CSV files are the same.")
    else:
        print(f"The files differ at {total_differences} positions.")
        print("Showing the first few differences:")
        for i, (row, col) in enumerate(differing_cells):
            if i >= max_differences:
                break
            print(f"Row {row}, Column '{col}': File1='{df1.at[row, col]}' vs File2='{df2.at[row, col]}'")
        if total_differences > max_differences:
            print(f"... and {total_differences - max_differences} more differences.")


compare_csv_files(file1, file2)


The files differ at 20505 positions.
Showing the first few differences:
Row 0, Column 'SchemeName': File1='40% Subsidy On Hank Yarn, Dyes & Chemicals Scheme' vs File2='Aponar Apon Ghar'
Row 0, Column 'Details': File1='DetailsThe "40% subsidy on Hank Yarn, Dyes & Chemicals Scheme" is a Subsidy Scheme by the Department of Handlooms & Textiles, Government of Andhra Pradesh. The entire assistance under the scheme will be in the form of Grant from the State Government. The scheme is operative from 29th April 2011. The subsidy will be available only on purchases / procurements from NHDC & APCO. The amount sanctioned will be credited directly to the members bank account of the concerned primary weavers cooperative societies.' vs File2='Details"Aponar Apon Ghar" is a Home Loan Subsidy Scheme by the Finance Department, Govt. of Assam. The objective is to realize the vision of “Housing For All” by 2022 so that each poor person may have their own house. All the applicants will get home loans at s

In [32]:
import pandas as pd

def sort_csv_by_first_column(input_file, output_file):
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Check if DataFrame is empty or has no columns
    if df.empty or df.shape[1] == 0:
        print("The CSV file is empty or does not have any columns.")
        return
    
    # Sort the DataFrame by the first column
    sorted_df = df.sort_values(by=df.columns[0])
    
    # Write the sorted DataFrame to a new CSV file
    sorted_df.to_csv(output_file, index=False)
    print(f"Sorted CSV saved to {output_file}")

# Example usage
input_file = 'scheme_sample2.csv'
output_file = 'sorted_scheme_sample2.csv'
sort_csv_by_first_column(input_file, output_file)


Sorted CSV saved to sorted_scheme_sample2.csv


In [51]:
#Headless mode

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.headless = True  # Run in headless mode
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

def extract_links_from_page():
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

# Open the initial URL
driver.get("https://www.myscheme.gov.in/search")

all_links_headless = []

try:
    while True:
        all_links_headless.extend(extract_links_from_page())
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)  # Wait for scrolling to complete
            next_button.click()
            time.sleep(2)  # Adjust the sleep time if needed to allow the next page to load properly
        except Exception as e:
            print("No more pages or error navigating to the next page. Stopping.")
            break
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the WebDriver
    driver.quit()

print(f"Scraped {len(all_links_headless)} links.")

No more pages or error navigating to the next page. Stopping.
Scraped 170 links.


Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Optimization by Scraping in a Single Session
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize WebDriver
def init_driver():
    options = webdriver.ChromeOptions()
    options.headless = True  # Run in headless mode
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    return driver, wait

def extract_links_from_page(driver, wait):
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

def scrape_links():
    driver, wait = init_driver()
    all_links = []

    try:
        driver.get("https://www.myscheme.gov.in/search")
        while True:
            all_links.extend(extract_links_from_page(driver, wait))
            try:
                next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(1)  # Wait for scrolling to complete
                next_button.click()
                time.sleep(2)  # Adjust the sleep time if needed to allow the next page to load properly
            except Exception as e:
                print("No more pages or error navigating to the next page. Stopping.")
                break
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the WebDriver
        driver.quit()

    print(f"Scraped {len(all_links)} links.")
    
    return all_links

async def fetch(session, url):
    try:
        async with session.get(url) as response:
            if response.status != 200:
                return None
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')

            scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
            detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
            benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
            eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
            exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
            application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
            documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
            faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
            keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
            
            return {
                'SchemeName': scheme_name,
                'Details': detail,
                'Benefits': benefit,
                'Eligibility': eligibility,
                'Exclusion': exclusion,
                'ApplicationProc': application_process,
                'DocumentsReq': documents_required,
                'FAQ': faq,
                'Link': url,
                'Keywords': keyword
            }
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

async def scrape_scheme_pages(links):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in links:
            tasks.append(fetch(session, url))
        results = await asyncio.gather(*tasks)
    return results

def update_dataset(new_links):
    # Load existing data if available
    if os.path.exists('scheme_sample.csv'):
        existing_data = pd.read_csv('scheme_sample.csv')
        existing_links = set(existing_data['Link'])
    else:
        existing_data = pd.DataFrame()
        existing_links = set()

    # Filter out new links
    unique_links = [link for link in new_links if link not in existing_links]

    # Scrape new data
    new_data = asyncio.run(scrape_scheme_pages(unique_links))
    
    # Filter out None results and create DataFrame
    new_data = [item for item in new_data if item]
    new_data_df = pd.DataFrame(new_data)

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df]).drop_duplicates(subset='Link', keep='last')
    else:
        combined_data = new_data_df

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample.csv', index=False)

    print("Dataset updated successfully.")

if __name__ == "__main__":
    new_links = scrape_links()
    update_dataset(new_links)


In [70]:
#WEB SCRAPING BASED ON LINKS
#If link already in csv, discard
#777 seconds 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

def setup_selenium(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.headless = True
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    return driver, wait

def extract_links_from_page(driver, wait):
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

def scrape_all_links(driver, wait, base_url):
    all_links = []
    driver.get(base_url)
    
    while True:
        all_links.extend(extract_links_from_page(driver, wait))
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)  # Wait for scrolling to complete
            next_button.click()
            time.sleep(2)  # Adjust the sleep time if needed to allow the next page to load properly
        except Exception as e:
            print("No more pages or error navigating to the next page. Stopping.")
            break
    
    driver.quit()
    print(f"Scraped {len(all_links)} links.")

    return all_links

def requests_session_with_retries(retries=3, backoff_factor=1):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(500, 502, 503, 504),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def scrape_scheme_page(session, url):
    try:
        response = session.get(url, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch {url} with status code {response.status_code}")
            return None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
        detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
        benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
        eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
        exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
        application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
        documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
        faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
        keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
        
        return {
            'SchemeName': scheme_name,
            'Details': detail,
            'Benefits': benefit,
            'Eligibility': eligibility,
            'Exclusion': exclusion,
            'ApplicationProc': application_process,
            'DocumentsReq': documents_required,
            'FAQ': faq,
            'Link': url,
            'Keywords': keyword
        }
    except requests.RequestException as e:
        print(f"Request error scraping {url}: {e}")
    except Exception as e:
        print(f"Error scraping {url}: {e}")
    return None

def main():
    base_url = "https://www.myscheme.gov.in/search"
    
    # Setup Selenium and extract links
    driver, wait = setup_selenium()
    all_links = scrape_all_links(driver, wait, base_url)
    
    # Load existing data if available
    if os.path.exists('scheme_sample3.csv'):
        existing_data = pd.read_csv('scheme_sample3.csv')
        existing_links = set(existing_data['Link'])
    else:
        existing_data = pd.DataFrame()
        existing_links = set()
    
    unique_links = list(set(all_links) - existing_links)
    
    # Initialize lists to store new data
    new_data = []

    # Create a requests session with retries
    session = requests_session_with_retries()

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scrape_scheme_page, session, url) for url in unique_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df]).drop_duplicates(subset='Link', keep='last')
    else:
        combined_data = new_data_df

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample3.csv', index=False)

    print("Scraping and updating complete.")

if __name__ == "__main__":
    main()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.89)
Stacktrace:
0   chromedriver                        0x00000001025fcee8 cxxbridge1$str$ptr + 1871728
1   chromedriver                        0x00000001025f554c cxxbridge1$str$ptr + 1840596
2   chromedriver                        0x000000010220882c cxxbridge1$string$len + 88532
3   chromedriver                        0x00000001021e3e5c core::str::slice_error_fail::he7b2aa4898bc357e + 3908
4   chromedriver                        0x0000000102271574 cxxbridge1$string$len + 517916
5   chromedriver                        0x0000000102283ddc cxxbridge1$string$len + 593796
6   chromedriver                        0x0000000102241478 cxxbridge1$string$len + 321056
7   chromedriver                        0x00000001022420e8 cxxbridge1$string$len + 324240
8   chromedriver                        0x00000001025c49fc cxxbridge1$str$ptr + 1641092
9   chr

In [63]:
#checks with contents of csv
#439 seconds
import time
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def setup_selenium(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.headless = True
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    return driver, wait

def extract_links_from_page(driver, wait):
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

def scrape_all_links(driver, wait, base_url):
    all_links = []
    driver.get(base_url)
    
    while True:
        all_links.extend(extract_links_from_page(driver, wait))
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)  # Wait for scrolling to complete
            next_button.click()
            time.sleep(2)  # Adjust the sleep time if needed to allow the next page to load properly
        except Exception as e:
            #print("No more pages or error navigating to the next page. Stopping.")
            break
    
    driver.quit()
    return all_links

def requests_session_with_retries(retries=3, backoff_factor=1):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(500, 502, 503, 504),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def scrape_scheme_page(session, url):
    try:
        response = session.get(url, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch {url} with status code {response.status_code}")
            return None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
        detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
        benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
        eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
        exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
        application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
        documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
        faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
        keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
        
        return {
            'SchemeName': scheme_name,
            'Details': detail,
            'Benefits': benefit,
            'Eligibility': eligibility,
            'Exclusion': exclusion,
            'ApplicationProc': application_process,
            'DocumentsReq': documents_required,
            'FAQ': faq,
            'Link': url,
            'Keywords': keyword
        }
    except requests.RequestException as e:
        print(f"Request error scraping {url}: {e}")
    except Exception as e:
        print(f"Error scraping {url}: {e}")
    return None

def main():
    base_url = "https://www.myscheme.gov.in/search"
    
    # Setup Selenium and extract links
    driver, wait = setup_selenium()
    all_links = scrape_all_links(driver, wait, base_url)
    
    # Load existing data if available
    if os.path.exists('scheme_sample4.csv'):
        existing_data = pd.read_csv('scheme_sample4.csv')
        existing_data['Combined'] = existing_data.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    else:
        existing_data = pd.DataFrame()
    
    unique_links = list(set(all_links))
    
    # Initialize lists to store new data
    new_data = []

    # Create a requests session with retries
    session = requests_session_with_retries()

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scrape_scheme_page, session, url) for url in unique_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)

    if not existing_data.empty:
        # Remove 'Combined' column from existing data
        existing_data = existing_data.drop(columns=['Combined'])
        
        # Update existing entries and add new entries
        for index, row in new_data_df.iterrows():
            link = row['Link']
            if link in existing_data['Link'].values:
                # Compare the new and existing data
                existing_row = existing_data[existing_data['Link'] == link].iloc[0]
                combined_existing = ' '.join(existing_row.values.astype(str))
                combined_new = ' '.join(row.values.astype(str))
                if combined_existing != combined_new:
                    # Update the row in existing data
                    existing_data.update(new_data_df.loc[index])
            else:
                # Add new entry
                existing_data = pd.concat([existing_data, new_data_df.loc[[index]]])
    else:
        existing_data = new_data_df

    # Save the updated data to CSV file
    existing_data.to_csv('scheme_sample4.csv', index=False)

    print("Scraping and updating complete.")

if __name__ == "__main__":
    main()


Failed to fetch https://www.myscheme.gov.in/schemes/elsnskfdch with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/fbascs with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/scsssetul with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/icms with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/amsmhsslc with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/madp with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/spy-gbocwwb with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/aamgsiscs with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/lels with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/makssy with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/jc with status code 429
Failed to fetch https://www.myscheme.gov.in/schemes/ehstgs with status code 429
Failed to fetch https://www.mysc

In [79]:
#480 seconds
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

def setup_selenium(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.headless = True
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    return driver, wait

def extract_links_from_page(driver, wait):
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

def scrape_all_links(driver, wait, base_url):
    all_links = []
    driver.get(base_url)
    
    while True:
        all_links.extend(extract_links_from_page(driver, wait))
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)  # Wait for scrolling to complete
            next_button.click()
            time.sleep(2)  # Adjust the sleep time if needed to allow the next page to load properly
        except Exception as e:
            print("No more pages or error navigating to the next page. Stopping.")
            break
    
    driver.quit()
    print(f"Scraped {len(all_links)} links.")

    return all_links

def requests_session_with_retries(retries=3, backoff_factor=1):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(500, 502, 503, 504),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def scrape_scheme_page(session, url):
    try:
        response = session.get(url, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch {url} with status code {response.status_code}")
            return None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
        detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
        benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
        eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
        exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
        application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
        documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
        faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
        keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
        
        return {
            'SchemeName': scheme_name,
            'Details': detail,
            'Benefits': benefit,
            'Eligibility': eligibility,
            'Exclusion': exclusion,
            'ApplicationProc': application_process,
            'DocumentsReq': documents_required,
            'FAQ': faq,
            'Link': url,
            'Keywords': keyword
        }
    except requests.RequestException as e:
        print(f"Request error scraping {url}: {e}")
    except Exception as e:
        print(f"Error scraping {url}: {e}")
    return None

def main():
    base_url = "https://www.myscheme.gov.in/search"
    
    # Setup Selenium and extract links
    driver, wait = setup_selenium()
    all_links = scrape_all_links(driver, wait, base_url)
    
    # Load existing data if available
    if os.path.exists('scheme_sample3.csv'):
        existing_data = pd.read_csv('scheme_sample3.csv')
        existing_links = set(existing_data['Link'])
    else:
        existing_data = pd.DataFrame()
        existing_links = set()
    
    unique_links = list(set(all_links) - existing_links)
    
    # Initialize lists to store new data
    new_data = []

    # Create a requests session with retries
    session = requests_session_with_retries()

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scrape_scheme_page, session, url) for url in unique_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df]).drop_duplicates()
    else:
        combined_data = new_data_df

    # Remove older rows if two rows have the same scheme name
    combined_data = combined_data.sort_values(by='Link').drop_duplicates(subset='SchemeName', keep='last')

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample3.csv', index=False)

    print("Scraping and updating complete.")

if __name__ == "__main__":
    main()


No more pages or error navigating to the next page. Stopping.
Scraped 10 links.
Scraping and updating complete.


In [ ]:
len(all_links)

In [84]:
# Taking a lot of time 1000+ seconds
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

def setup_selenium(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.headless = True
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    return driver, wait

def extract_links_from_page(driver, wait):
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

def scrape_all_links(driver, wait, base_url):
    all_links = []
    driver.get(base_url)
    
    while True:
        all_links.extend(extract_links_from_page(driver, wait))
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)  # Wait for scrolling to complete
            next_button.click()
            time.sleep(2)  # Adjust the sleep time if needed to allow the next page to load properly
        except Exception as e:
            print("No more pages or error navigating to the next page. Stopping.")
            break
    
    driver.quit()
    print(f"Scraped {len(all_links)} links.")

    return all_links

def requests_session_with_retries(retries=3, backoff_factor=1):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(429, 500, 502, 503, 504),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def scrape_scheme_page(session, url):
    retries = 5
    delay = 1  # initial delay in seconds
    for i in range(retries):
        try:
            response = session.get(url, timeout=10)
            if response.status_code == 429:
                print(f"Rate limit hit. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # exponential backoff
                continue
            elif response.status_code != 200:
                print(f"Failed to fetch {url} with status code {response.status_code}")
                return None
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
            detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
            benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
            eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
            exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
            application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
            documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
            faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
            keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
            
            return {
                'SchemeName': scheme_name,
                'Details': detail,
                'Benefits': benefit,
                'Eligibility': eligibility,
                'Exclusion': exclusion,
                'ApplicationProc': application_process,
                'DocumentsReq': documents_required,
                'FAQ': faq,
                'Link': url,
                'Keywords': keyword
            }
        except requests.RequestException as e:
            print(f"Request error scraping {url}: {e}")
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    return None

def main():
    base_url = "https://www.myscheme.gov.in/search"
    
    # Setup Selenium and extract links
    driver, wait = setup_selenium()
    all_links = scrape_all_links(driver, wait, base_url)

    seen = set()
    unique_links = []
    for item in all_links:
        if item not in seen:
            unique_links.append(item)
            seen.add(item)

    print(f"Unique {len(unique_links)} links.")

    all_links = unique_links
    
    # Load existing data if available
    if os.path.exists('scheme_sample3.csv'):
        existing_data = pd.read_csv('scheme_sample3.csv')
    else:
        existing_data = pd.DataFrame()
    
    # Initialize lists to store new data
    new_data = []

    # Create a requests session with retries
    session = requests_session_with_retries()

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scrape_scheme_page, session, url) for url in all_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)
                time.sleep(random.uniform(1, 3))  # Random sleep to prevent hitting rate limits

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df])
    else:
        combined_data = new_data_df

    # Remove duplicates based on 'Link' and keep the latest
    combined_data = combined_data.drop_duplicates(subset=['Link'], keep='last')
    
    # Remove duplicates based on 'SchemeName' and keep the latest
    combined_data = combined_data.sort_values('Link').drop_duplicates(subset=['SchemeName'], keep='last')

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample3.csv', index=False)

    print("Scraping and updating complete.")

if __name__ == "__main__":
    main()


No more pages or error navigating to the next page. Stopping.
Scraped 2449 links.
Unique 2432 links.


KeyboardInterrupt: 

In [91]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

def setup_selenium(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.headless = True
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    return driver, wait

def extract_links_from_page(driver, wait):
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

def scrape_all_links(driver, wait, base_url):
    all_links = []
    driver.get(base_url)
    
    while True:
        all_links.extend(extract_links_from_page(driver, wait))
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)  # Wait for scrolling to complete
            next_button.click()
            time.sleep(2)  # Adjust the sleep time if needed to allow the next page to load properly
        except Exception as e:
            print("No more pages or error navigating to the next page. Stopping.")
            break
    
    driver.quit()
    print(f"Scraped {len(all_links)} links.")

    return all_links

def requests_session_with_retries(retries=3, backoff_factor=1):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(429, 500, 502, 503, 504),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def scrape_scheme_page(session, url):
    retries = 5
    delay = 1  # initial delay in seconds
    for i in range(retries):
        try:
            response = session.get(url, timeout=10)
            if response.status_code == 429:
                print(f"Rate limit hit. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # exponential backoff
                continue
            elif response.status_code != 200:
                print(f"Failed to fetch {url} with status code {response.status_code}")
                return None
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
            detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
            benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
            eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
            exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
            application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
            documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
            faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
            keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
            
            return {
                'SchemeName': scheme_name,
                'Details': detail,
                'Benefits': benefit,
                'Eligibility': eligibility,
                'Exclusion': exclusion,
                'ApplicationProc': application_process,
                'DocumentsReq': documents_required,
                'FAQ': faq,
                'Link': url,
                'Keywords': keyword
            }
        except requests.RequestException as e:
            print(f"Request error scraping {url}: {e}")
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    return None

def main():
    base_url = "https://www.myscheme.gov.in/search"
    
    # Setup Selenium and extract links
    driver, wait = setup_selenium()
    all_links = scrape_all_links(driver, wait, base_url)

    seen = set()
    unique_links = []
    for item in all_links:
        if item not in seen:
            unique_links.append(item)
            seen.add(item)

    print(f"Unique {len(unique_links)} links.")

    all_links = unique_links
    
    # Load existing data if available
    if os.path.exists('scheme_sample3.csv'):
        existing_data = pd.read_csv('scheme_sample3.csv')
    else:
        existing_data = pd.DataFrame()
    
    # Initialize lists to store new data
    new_data = []

    # Create a requests session with retries
    session = requests_session_with_retries()

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scrape_scheme_page, session, url) for url in all_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)
                time.sleep(random.uniform(1, 3))  # Random sleep to prevent hitting rate limits

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df])
    else:
        combined_data = new_data_df

    # Remove duplicates based on 'Link' and keep the latest
    combined_data = combined_data.drop_duplicates(subset=['Link'], keep='last')
    
    # Remove duplicates based on 'SchemeName' and keep the latest
    combined_data = combined_data.sort_values('Link').drop_duplicates(subset=['SchemeName'], keep='last')

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample3.csv', index=False)

    print("Scraping and updating complete.")

if __name__ == "__main__":
    main()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.89)
Stacktrace:
0   chromedriver                        0x0000000100a4cee8 cxxbridge1$str$ptr + 1871728
1   chromedriver                        0x0000000100a4554c cxxbridge1$str$ptr + 1840596
2   chromedriver                        0x000000010065882c cxxbridge1$string$len + 88532
3   chromedriver                        0x0000000100633e5c core::str::slice_error_fail::he7b2aa4898bc357e + 3908
4   chromedriver                        0x00000001006c1574 cxxbridge1$string$len + 517916
5   chromedriver                        0x00000001006d3ddc cxxbridge1$string$len + 593796
6   chromedriver                        0x0000000100691478 cxxbridge1$string$len + 321056
7   chromedriver                        0x00000001006920e8 cxxbridge1$string$len + 324240
8   chromedriver                        0x0000000100a149fc cxxbridge1$str$ptr + 1641092
9   chr

In [109]:
# works well but takes 30 minutes
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

def setup_selenium(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.headless = True
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    return driver, wait

def extract_links_from_page(driver, wait):
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

def scrape_all_links(driver, wait, base_url):
    all_links = []
    driver.get(base_url)
    
    while True:
        all_links.extend(extract_links_from_page(driver, wait))
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(0.5)  # Wait for scrolling to complete
            next_button.click()
            time.sleep(1.5)  # Adjust the sleep time if needed to allow the next page to load properly
        except Exception as e:
            print("No more pages or error navigating to the next page. Stopping.")
            break
    
    driver.quit()
    print(f"Scraped {len(all_links)} links.")

    return all_links

def requests_session_with_retries(retries=3, backoff_factor=1):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(429, 500, 502, 503, 504),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def scrape_scheme_page(session, url):
    retries = 5
    delay = 1  # initial delay in seconds
    for i in range(retries):
        try:
            response = session.get(url, timeout=10)
            if response.status_code == 429:
                print(f"Rate limit hit. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # exponential backoff
                continue
            elif response.status_code != 200:
                print(f"Failed to fetch {url} with status code {response.status_code}")
                return None
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
            detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
            benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
            eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
            exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
            application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
            documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
            faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
            keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
            
            return {
                'SchemeName': scheme_name,
                'Details': detail,
                'Benefits': benefit,
                'Eligibility': eligibility,
                'Exclusion': exclusion,
                'ApplicationProc': application_process,
                'DocumentsReq': documents_required,
                'FAQ': faq,
                'Link': url,
                'Keywords': keyword
            }
        except requests.RequestException as e:
            print(f"Request error scraping {url}: {e}")
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    return None

def main():
    base_url = "https://www.myscheme.gov.in/search"
    
    # Setup Selenium and extract links
    driver, wait = setup_selenium()
    all_links = scrape_all_links(driver, wait, base_url)

    seen = set()
    unique_links = []
    for item in all_links:
        if item not in seen:
            unique_links.append(item)
            seen.add(item)

    print(f"Unique {len(unique_links)} links.")

    all_links = unique_links

    all_links.sort()
    
    # Load existing data if available
    if os.path.exists('scheme_sample3.csv'):
        existing_data = pd.read_csv('scheme_sample3.csv')
    else:
        existing_data = pd.DataFrame()
    
    # Initialize lists to store new data
    new_data = []

    # Create a requests session with retries
    session = requests_session_with_retries()

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scrape_scheme_page, session, url) for url in all_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)
                print(f"Scraped data from {result['Link']}")
                time.sleep(random.uniform(1, 3))  # Random sleep to prevent hitting rate limits

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)
    print(f"Scraped data for {len(new_data_df)} schemes.")

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df])
    else:
        combined_data = new_data_df

    # Remove duplicates and keep the latest
    combined_data = combined_data.drop_duplicates(keep='last')
    
    # Remove duplicates based on 'SchemeName' and keep the latest
    combined_data = combined_data.sort_values('Link').drop_duplicates(subset=['SchemeName'], keep='last')

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample3.csv', index=False)
    print("Scraping and updating complete. Data saved to scheme_sample3.csv.")

if __name__ == "__main__":
    main()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.89)
Stacktrace:
0   chromedriver                        0x00000001044b8ee8 cxxbridge1$str$ptr + 1871728
1   chromedriver                        0x00000001044b154c cxxbridge1$str$ptr + 1840596
2   chromedriver                        0x00000001040c482c cxxbridge1$string$len + 88532
3   chromedriver                        0x000000010409fe5c core::str::slice_error_fail::he7b2aa4898bc357e + 3908
4   chromedriver                        0x000000010412d574 cxxbridge1$string$len + 517916
5   chromedriver                        0x000000010413fddc cxxbridge1$string$len + 593796
6   chromedriver                        0x00000001040fd478 cxxbridge1$string$len + 321056
7   chromedriver                        0x00000001040fe0e8 cxxbridge1$string$len + 324240
8   chromedriver                        0x00000001044809fc cxxbridge1$str$ptr + 1641092
9   chr

In [113]:
#2190 seconds

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

def setup_selenium(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.headless = True
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    return driver, wait
    
def extract_links_from_page(driver, wait):
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

def scrape_all_links(driver, wait, base_url):
    all_links = []
    driver.get(base_url)
    
    while True:
        all_links.extend(extract_links_from_page(driver, wait))
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(0.5)  # Wait for scrolling to complete
            next_button.click()
            time.sleep(1.5)  # Adjust the sleep time if needed to allow the next page to load properly
        except Exception as e:
            #print("No more pages or error navigating to the next page. Stopping.")
            break
    
    driver.quit()
    print(f"Scraped {len(all_links)} links.")
    return list(set(all_links))  # Ensure uniqueness

def requests_session_with_retries(retries=3, backoff_factor=1):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(429, 500, 502, 503, 504),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def scrape_scheme_page(session, url):
    retries = 5
    delay = 1  # initial delay in seconds
    for i in range(retries):
        try:
            response = session.get(url, timeout=10)
            if response.status_code == 429:
                print(f"Rate limit hit. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # exponential backoff
                continue
            elif response.status_code != 200:
                print(f"Failed to fetch {url} with status code {response.status_code}")
                return None
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
            detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
            benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
            eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
            exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
            application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
            documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
            faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
            keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
            
            return {
                'SchemeName': scheme_name,
                'Details': detail,
                'Benefits': benefit,
                'Eligibility': eligibility,
                'Exclusion': exclusion,
                'ApplicationProc': application_process,
                'DocumentsReq': documents_required,
                'FAQ': faq,
                'Link': url,
                'Keywords': keyword
            }
        except requests.RequestException as e:
            print(f"Request error scraping {url}: {e}")
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    return None

def main():
    base_url = "https://www.myscheme.gov.in/search"
    
    # Setup Selenium and extract links
    driver, wait = setup_selenium()
    all_links = scrape_all_links(driver, wait, base_url)

    print(f"Unique {len(all_links)} links.")

    # Load existing data if available
    if os.path.exists('scheme_sample3.csv'):
        existing_data = pd.read_csv('scheme_sample3.csv')
    else:
        existing_data = pd.DataFrame()
    
    # Initialize lists to store new data
    new_data = []

    # Create a requests session with retries
    session = requests_session_with_retries()

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=20) as executor:  # Increased workers for faster scraping
        futures = [executor.submit(scrape_scheme_page, session, url) for url in all_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)
                print(f"Scraped data from {result['Link']}")
                time.sleep(random.uniform(0.5, 1.5))  # Reduced sleep to speed up processing

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)
    print(f"Scraped data for {len(new_data_df)} schemes.")

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df])
    else:
        combined_data = new_data_df

    # Remove duplicates and keep the latest
    combined_data = combined_data.drop_duplicates(keep='last')
    
    # Remove duplicates based on 'SchemeName' and keep the latest
    combined_data = combined_data.sort_values('Link').drop_duplicates(subset=['SchemeName'], keep='last')

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample3.csv', index=False)
    print("Scraping and updating complete. Data saved to scheme_sample3.csv.")

if __name__ == "__main__":
    main()


Scraped 2439 links.
Unique 2422 links.
Scraped data from https://www.myscheme.gov.in/schemes/wbedrj
Scraped data from https://www.myscheme.gov.in/schemes/fafeoscob
Scraped data from https://www.myscheme.gov.in/schemes/hhjmhhst
Scraped data from https://www.myscheme.gov.in/schemes/ral
Scraped data from https://www.myscheme.gov.in/schemes/svpmss
Scraped data from https://www.myscheme.gov.in/schemes/bhsstg
Scraped data from https://www.myscheme.gov.in/schemes/stsemstsmcupscviii
Scraped data from https://www.myscheme.gov.in/schemes/kdky
Scraped data from https://www.myscheme.gov.in/schemes/cbsp-ep
Scraped data from https://www.myscheme.gov.in/schemes/dbanbocwwb
Scraped data from https://www.myscheme.gov.in/schemes/tlnskfdch
Scraped data from https://www.myscheme.gov.in/schemes/e-yuvasbirace-yuvafugs
Scraped data from https://www.myscheme.gov.in/schemes/sitviiisbwodap
Scraped data from https://www.myscheme.gov.in/schemes/tloiis
Scraped data from https://www.myscheme.gov.in/schemes/ssvy
Scra

In [114]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

def setup_selenium(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    return driver, wait

def extract_links_from_page(driver, wait):
    links = []
    divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.p-4.lg\\:p-8.w-full")))
    for div in divs:
        try:
            a_tag = div.find_element(By.XPATH, ".//h2/a")
            link = a_tag.get_attribute("href")
            if link:
                links.append(link)
        except Exception as e:
            print(f"Error finding link in div: {e}")
    return links

def scrape_all_links(driver, wait, base_url):
    all_links = []
    driver.get(base_url)
    
    while True:
        all_links.extend(extract_links_from_page(driver, wait))
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ml-2.text-darkblue-900.dark\\:text-white.cursor-pointer")))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(0.5)  # Wait for scrolling to complete
            next_button.click()
            time.sleep(1.5)  # Adjust the sleep time if needed to allow the next page to load properly
        except Exception as e:
            #print("No more pages or error navigating to the next page. Stopping.")
            break
    
    driver.quit()
    print(f"Scraped {len(all_links)} links.")
    return list(set(all_links))  # Ensure uniqueness

def requests_session_with_retries(retries=3, backoff_factor=1):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=(429, 500, 502, 503, 504),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def scrape_scheme_page(session, url):
    retries = 5
    delay = 1  # initial delay in seconds
    for i in range(retries):
        try:
            response = session.get(url, timeout=10)
            if response.status_code == 429:
                print(f"Rate limit hit. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # exponential backoff
                continue
            elif response.status_code != 200:
                print(f"Failed to fetch {url} with status code {response.status_code}")
                return None
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            scheme_name = soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white").text.strip() if soup.find('h1', class_="font-bold text-xl sm:text-2xl mt-1 text-[#24262B] dark:text-white") else 'N/A'
            detail = soup.find('div', id='details').text.strip() if soup.find('div', id='details') else 'N/A'
            benefit = soup.find('div', id='benefits').text.strip() if soup.find('div', id='benefits') else 'N/A'
            eligibility = soup.find('div', id='eligibility').text.strip() if soup.find('div', id='eligibility') else 'N/A'
            exclusion = soup.find('div', id='exclusions').text.strip() if soup.find('div', id='exclusions') else 'N/A'
            application_process = soup.find('div', id='application-process').text.strip() if soup.find('div', id='application-process') else 'N/A'
            documents_required = soup.find('div', id='documents-required').text.strip() if soup.find('div', id='documents-required') else 'N/A'
            faq = soup.find('div', id='faqs').text.strip() if soup.find('div', id='faqs') else 'N/A'
            keyword = soup.find('div', class_='mb-2 md:mb-0 w-full').text.strip() if soup.find('div', class_='mb-2 md:mb-0 w-full') else 'N/A'
            
            return {
                'SchemeName': scheme_name,
                'Details': detail,
                'Benefits': benefit,
                'Eligibility': eligibility,
                'Exclusion': exclusion,
                'ApplicationProc': application_process,
                'DocumentsReq': documents_required,
                'FAQ': faq,
                'Link': url,
                'Keywords': keyword
            }
        except requests.RequestException as e:
            print(f"Request error scraping {url}: {e}")
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    return None

def main():
    base_url = "https://www.myscheme.gov.in/search"    
    # Setup Selenium and extract links
    driver, wait = setup_selenium()
    all_links = scrape_all_links(driver, wait, base_url)

    print(f"Unique {len(all_links)} links.")

    # Load existing data if available
    if os.path.exists('scheme_sample3.csv'):
        existing_data = pd.read_csv('scheme_sample3.csv')
    else:
        existing_data = pd.DataFrame()
    
    # Initialize lists to store new data
    new_data = []

    # Create a requests session with retries
    session = requests_session_with_retries()

    # Use ThreadPoolExecutor to manage concurrent scraping
    with ThreadPoolExecutor(max_workers=20) as executor:  # Increased workers for faster scraping
        futures = [executor.submit(scrape_scheme_page, session, url) for url in all_links]
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                new_data.append(result)
                print(f"Scraped data from {result['Link']}")
                time.sleep(random.uniform(0.5, 1.5))  # Reduced sleep to speed up processing

    # Create a pandas DataFrame for new data
    new_data_df = pd.DataFrame(new_data)
    print(f"Scraped data for {len(new_data_df)} schemes.")

    # Combine new data with existing data
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, new_data_df])
    else:
        combined_data = new_data_df

    # Remove duplicates and keep the latest
    combined_data = combined_data.drop_duplicates(keep='last')
    
    # Remove duplicates based on 'SchemeName' and keep the latest
    combined_data = combined_data.sort_values('Link').drop_duplicates(subset=['SchemeName'], keep='last')

    # Save the combined data to CSV file
    combined_data.to_csv('scheme_sample3.csv', index=False)
    print("Scraping and updating complete. Data saved to scheme_sample3.csv.")

if __name__ == "__main__":
    main()


1
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
Scraped 2439 links.
Unique 2432 links.
Scraped data from https://www.myscheme.gov.in/schemes/dbanbocwwb
Scraped data from https://www.myscheme.gov.in/schemes/apwesmc
Scraped data from https://www.myscheme.gov.in/schemes/hhjmhhst
Scraped data from https://www.myscheme.gov.in/schemes/sitviiisbwodap
Scraped data from https://www.myscheme.gov.in/schemes/wbedrj
Scraped data from https://www.myscheme.gov.in/schemes/svpmss
Scraped data from https://www.myscheme.gov.in/schemes/bse-pg
Scraped data from https://www